# Generate Schemas

## Setup

In [1]:
#!fsharp
#r "nuget: FSharp.Data"
#r "./src/Reports/bin/Debug/net5.0/Reports.dll"

open FSharp.Data
open OSTOR.ClinicalTrials.Reports.Utilities
open OSTOR.ClinicalTrials.Reports.Utilities.Xml

open System
open System.IO

open System.Xml.Linq

// [<Literal>]
// let fmiXsdPath = "./src/Reports/data/FMI/fmiReport0.xsd"
// [<Literal>]
// let fmiXsdFolder = "./src/Reports/data/FMI/"
// type FmiXml = XmlProvider<Sample = fmiXmlsPath, SampleIsList = true>

// let fmiXml = FmiXml.Load("./data/FMI/ORD-0931199-01.xml")

// query {
//     for element in fmiXml.XElement.Elements() do
//     where (element.Name.LocalName.Equals("CustomerInformation"))
//     select (element.)
// }
// fmiXml.XElement.Element(XName.Get "rr:ResultsReport")

[<Literal>]
let fmiXmlsPath = "./data/FMI/ORD-0758611-01.xml"
let xmlText = File.ReadAllText(fmiXmlsPath)

let rrNs = XNamespace.Get "http://integration.foundationmedicine.com/reporting"
let xml = XDocument.Parse(xmlText)


let elem = Seq.head (xml.Element(rrNs + "ResultsReport").Element(rrNs + "CustomerInformation").Elements())

// query {
//     for element in xml.Element(rrNs + "ResultsReport").Element(rrNs + "CustomerInformation").Elements() do
//     select {| MRN = element.Element(rrNs + "MRN") |}
// }

let resultsReportElement = xml.Element(rrNs + "ResultsReport")
let customerInfoElement = resultsReportElement.Element(rrNs + "CustomerInformation")
let mrn = customerInfoElement.Element(rrNs + "MRN").Value

let finalReportElement = resultsReportElement.Element(rrNs + "ResultsPayload").Element(XName.Get "FinalReport")
let reportId = finalReportElement.Element(XName.Get "ReportId").Value
let sampleName = finalReportElement.Element(XName.Get "SampleName").Value
sampleName


US0784680.01

## Caris Schema

In [1]:
#!fsharp
let carisSchemaGenerator = XmlSchemaGenerator()

// Caris report xmls
let carisReportXmls = Path.Combine([|Environment.CurrentDirectory; "data"; "Caris"|])

DirectoryInfo(carisReportXmls).EnumerateFileSystemInfos("*.xml")
    |> Seq.iter (fun fileInfo -> carisSchemaGenerator.AddXml(fileInfo.FullName))

carisSchemaGenerator.SaveSchemaToDisk "./src/Reports/data/carisReport.xsd"

## FMI

In [1]:
#!fsharp
let fmiSchemaGenerator = XmlSchemaGenerator()

// FMI report xmls
let fmiReportXmls = Path.Combine([|Environment.CurrentDirectory; "data"; "FMI"|])

DirectoryInfo(fmiReportXmls).EnumerateFileSystemInfos("*.xml")
    |> Seq.iter (fun dirInfo ->
        String.readFromDisk dirInfo.FullName
        // |> Regex.remove " xsi:schemaLocation=\".+\""
        // |> Regex.remove " xsi:noNamespaceSchemaLocation=\".+\""
        |> String.writeToDisk $"./data/parsed/FMI/{dirInfo.Name}"
    )

let parsedFmiReportXmls = Path.Combine([|Environment.CurrentDirectory; "data"; "parsed"; "FMI"|])

let sg =
    DirectoryInfo(parsedFmiReportXmls).EnumerateFileSystemInfos("*.xml")
    |> Seq.fold (fun (sg: XmlSchemaGenerator) dirInfo ->
        sg.AddXml(dirInfo.FullName)
    ) fmiSchemaGenerator

sg.SchemaStrings |> Seq.iteri (fun num schemaString -> File.WriteAllText($"./src/Reports/data/fmiReport{num}.xsd", schemaString))